# Obtendo precisões de acertos reais do algoritmo
#### Usando o método 'StratifiedKFold' de validação cruzada na base de dados para visualizar a precisão de acertos do algoritmo.

### Pré-processamento da base de dados

In [1]:
# importando a biblioteca pandas do python
import pandas as pd

# importando a base de dados para o objeto "dataframe"
dataframe = pd.read_csv('Dados de Credito.csv', encoding = 'utf-8', sep = ',')

# substituindo valores inconsistentes no campo idades pela média das idades consistentes
dataframe.loc[dataframe.age < 0, 'age'] = 40.92

# separando os atributos previsores do meta classe
previsores = dataframe.iloc[:, 1:4].values
classe = dataframe.iloc[:, 4].values

# importando a biblioteca sklearn do python
from sklearn.impute import SimpleImputer
# função "SimpleImputer" responsável por corrigir valores faltantes na base de dados

# importando a biblioteca numpy do python
import numpy as np

# criando o objeto "imputer"
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")

# fazendo o treinamento com a base de dados para correção de valores faltantes
imputer = imputer.fit(previsores[:,0:3])

# corrigindo os valores faltantes usando o objeto criado "imputer"
previsores[:, 0:3] = imputer.transform(previsores[:, 0:3])

# importando a biblioteca sklearn do python
from sklearn.preprocessing import StandardScaler
# a função "StandardScaler" tem a função de escalonas toda a base de dados para corrigir a discrepância
# dos valores

# criando o objeto "scaler"
scaler = StandardScaler()

# escalonando todos os atributos para auxiliar no cálculo de distâncias euclidianas
previsores = scaler.fit_transform(previsores)

> Nesse caso, não será preciso separar a base de testes em um modelo de treinamento e outro de teste. O própio algoritmo irá fazer isso automaticamente em N porções informadas, usando diversos testes com essas porções.

### 'StratifiedKFold' no Python

Para testar esse modelo de validação cruzada, vamos usar o algoritmo naive bayes para fazer a previsão na base de dados, dado que a execução é mais rápida. Entretanto, qualquer algoritmo de Machine Learning pode ser usado.

In [2]:
# importando a biblioteca sklearn do python
from sklearn.model_selection import StratifiedKFold
# 'cross_val_score' aplica o método de validação cruzada que possui esse mesmo nome

# importando a biblioteca sklearn do python
from sklearn.naive_bayes import GaussianNB
# 'GaussianNB' aplica o método do algoritmo naive bayes na base de dados

# importando a biblioteca sklearn do python
from sklearn.metrics import accuracy_score, confusion_matrix
# 'accuracy_score' responsável por determinar a precisão de acertos do algoritmo
# 'confusion_matrix' cria uma matriz de confusão para a previsão feita no algoritmo

# importando a biblioteca numpy do python
import numpy as np

In [3]:
# configurando o objeto 'kfold' para realizar a validação cruzada
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 0)
# 'n_splits' indica a quantidade de fracionamentos que será realizada na base de dados
# 'shuffle' garante a aleatoriedade do fracionamento na base de dados
# 'random_state' define uma semente geradora para os dados pseudo-aleatórios

In [4]:
# criando uma lista para armazer os resultados obtidos
resultados = []
# criando uma lista para armazenar as matrizes de confusão obtidas
matrizes = []

In [5]:
# criando uma estrutura de reptição for para verificar como a base de dados é fracionada em cada execução
for indice_treinamento, indice_teste in kfold.split(previsores, np.zeros(shape = (previsores.shape[0], 1))):
    print('Indice de Treinamento: {}'.format(indice_treinamento))
    print('Indice de Teste: {}'.format(indice_teste))

Indice de Treinamento: [   0    1    2 ... 1997 1998 1999]
Indice de Teste: [   9   13   37   40   43   52   64   76   77   81   90   92   97  104
  117  119  121  123  131  144  147  154  172  173  231  249  255  261
  283  289  290  310  312  322  323  329  337  339  347  350  358  361
  366  379  382  391  401  410  444  465  485  490  512  533  534  555
  559  561  565  568  569  578  596  599  617  619  625  634  655  677
  690  700  708  719  723  730  736  739  740  769  782  784  794  799
  819  868  878  885  890  897  929  933  939  943  980  981 1000 1002
 1007 1010 1019 1022 1042 1052 1055 1059 1065 1066 1082 1104 1124 1140
 1149 1165 1170 1174 1179 1182 1191 1197 1199 1216 1221 1224 1233 1243
 1246 1249 1262 1266 1267 1268 1289 1310 1328 1362 1372 1391 1417 1436
 1446 1461 1469 1470 1492 1497 1502 1510 1513 1526 1552 1554 1613 1614
 1627 1628 1640 1644 1661 1667 1675 1677 1683 1684 1685 1690 1693 1697
 1705 1714 1750 1751 1758 1759 1765 1773 1785 1787 1798 1803 1813 1830
 

Verifique que a base de dados foi fracionada 10 vezes, e para cada vez são escolhidos registros aleatórios. a diferença desse método para o 'cross_val_score' é que é possível repetir valores em diferentes base de dados fracionadas, sempre de forma aleatória.

In [6]:
# criando uma estrutura de reptição for para obter a precisão de cada base de dados fracionada
for indice_treinamento, indice_teste in kfold.split(previsores, np.zeros(shape = (previsores.shape[0], 1))):
    # criando o objeto 'classificador' responsável por gerar a tabela de probabilidades pelo método naive bayes
    classificador = GaussianNB()
    # realizando o treinamento com cada base de dados fracionada de forma randômica
    classificador.fit(previsores[indice_treinamento], classe[indice_treinamento])
    # obtendo a capacidade de previsão de algoritmos usando as bases de teste fracionada
    previsoes = classificador.predict(previsores[indice_teste])
    # obtendo a precisão de acertos do algoritmo para cada base de dados de teste fracionada
    precisao = accuracy_score(classe[indice_teste], previsoes)
    # obtendo as matrizes de confusão e armazenando na lista 'matrizes'
    matrizes.append(confusion_matrix(classe[indice_teste], previsoes))
    # adicionando o valor da precisão na lista 'resultados'
    resultados.append(precisao)

In [7]:
# convertendo a lista para um array numpy
resultados = np.asarray(resultados)

In [8]:
# visualizando a porcentagem de acertos para cada um dos fracionamentos feitos na base de dados original
print(resultados)

[0.945 0.905 0.925 0.935 0.91  0.935 0.94  0.925 0.93  0.905]


In [9]:
# observando um resultado geral para a base de dados usando a média dos dados
resultados.mean()

0.9254999999999999

Desse modo, é mais preciso dizer que a porcentagem de acertos pelo algoritmo com a base de dados usada é **92.4%**, pois é um valor mais preciso do que usar um modelo somente para teste.

In [10]:
# observando o quanto os dados se desviaram dos valores necessários de serem obtidos
resultados.std()

0.013683932183403994

Com o desvio padrão, é possível visualizar o quão os dados obtidos estão distantes do resultado correto, quanto maior for o valor obtido, mais inconsistente é a precisão do algoritmo para uma determinada base de dados.

In [11]:
# observando a matriz de confusão obtida para o primeiro fracionamento feito
print(matrizes[0], '\n')
# observando a matriz de confusão obtida para o segundo fracionamento feito
print(matrizes[1], '\n')
# observando a matriz de confusão obtida para o terceiro fracionamento feito
print(matrizes[2], '\n')
# observando a matriz de confusão obtida para o quarto fracionamento feito
print(matrizes[3], '\n')
# observando a matriz de confusão obtida para o quinto fracionamento feito
print(matrizes[4], '\n')
# observando a matriz de confusão obtida para o sexto fracionamento feito
print(matrizes[5], '\n')
# observando a matriz de confusão obtida para o sétimo fracionamento feito
print(matrizes[6], '\n')
# observando a matriz de confusão obtida para o oitavo fracionamento feito
print(matrizes[7], '\n')
# observando a matriz de confusão obtida para o nono fracionamento feito
print(matrizes[8], '\n')
# observando a matriz de confusão obtida para o décimo fracionamento feito
print(matrizes[9], '\n')

[[179   3]
 [  8  10]] 

[[168   6]
 [ 13  13]] 

[[170   4]
 [ 11  15]] 

[[167   2]
 [ 11  20]] 

[[163   5]
 [ 13  19]] 

[[172   5]
 [  8  15]] 

[[171   2]
 [ 10  17]] 

[[160   1]
 [ 14  25]] 

[[170   2]
 [ 12  16]] 

[[160   7]
 [ 12  21]] 



In [12]:
# obtendo uma média dos resultados para a matriz de confusão
matriz_final = np.mean(matrizes, axis = 0)

In [13]:
# visualizando a matriz de confusão geral
print(matriz_final)

[[168.    3.7]
 [ 11.2  17.1]]


Desse modo, essa é a matriz de confusão mais correta em relação as outras obtidas, pois ela levou em consideração processos estatísticos mais seguros de se confiar.

### Alguma dúvida? Entre em contato comigo:

- [Me envie um e-mail](mailto:alyssonmachado388@gmail.com);